In [1]:
!pip install ultralytics -q 

In [2]:
# Nhập các thư viện cần thiết cho dự án
import os  # Thư viện để tương tác với hệ thống tệp
import pickle  # Thư viện để lưu trữ và tải dữ liệu Python dưới dạng nhị phân
import numpy as np  # Thư viện cho tính toán số học với mảng n-dimensional
import pandas as pd  # Thư viện để xử lý và phân tích dữ liệu
from tqdm.notebook import tqdm  # Thư viện để hiển thị thanh tiến trình trong các vòng lặp
from PIL import Image  # Thư viện để xử lý hình ảnh
import requests  # Thư viện để gửi các yêu cầu HTTP
from io import BytesIO  # Cung cấp một cách để làm việc với dữ liệu nhị phân trong bộ nhớ

# Nhập các thư viện TensorFlow và Keras cho học sâu
import tensorflow  # Thư viện cho học sâu
from tensorflow import keras  # API cao cấp cho TensorFlow
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input  # Mô hình VGG16 và hàm tiền xử lý
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Hàm để tải và chuyển đổi hình ảnh
from tensorflow.keras.preprocessing.text import Tokenizer  # Dùng để chuyển đổi văn bản thành các số nguyên (token)
from tensorflow.keras.preprocessing.sequence import pad_sequences  # Hàm để điều chỉnh độ dài của các chuỗi đầu vào
from tensorflow.keras.models import Model, load_model  # Các lớp để xây dựng và tải mô hình Keras
from tensorflow.keras.utils import to_categorical, plot_model  # Hàm để chuyển đổi nhãn thành dạng one-hot encoding và vẽ mô hình
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add, Flatten  # Các lớp để xây dựng mô hình mạng nơ-ron

# Nhập các thư viện từ Transformers cho xử lý ngôn ngữ tự nhiên
from transformers import BertTokenizer, BertModel  # Dùng để mã hóa văn bản cho mô hình BERT

# Nhập các thư viện từ Ultralytics cho phát hiện đối tượng
from ultralytics.nn.tasks import DetectionModel  # Lớp để xây dựng mô hình phát hiện đối tượng
from ultralytics.nn.tasks import attempt_load_one_weight  # Hàm để tải trọng số cho mô hình
from ultralytics import YOLO  # Mô hình YOLO để phát hiện đối tượng

# Nhập các thư viện PyTorch và OpenCV cho xử lý hình ảnh
import torch  # Thư viện cho học sâu với PyTorch
import cv2  # Thư viện OpenCV để xử lý hình ảnh
from torchvision import transforms  # Các hàm để thực hiện các phép biến đổi trên hình ảnh trong PyTorch

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
BASE_DIR = '/kaggle/input/dataset'
WORKING_DIR = '/kaggle/working/'

In [4]:
# Tải mô hình YOLO
model = YOLO('yolov8n.pt') 

100%|██████████| 6.25M/6.25M [00:00<00:00, 105MB/s]


In [5]:
# Đọc ảnh bằng OpenCV
image_path = "/kaggle/input/test-data-encode-25-11-2024/boy_cycle.jpg"  # Đường dẫn đến ảnh cần đọc
image = cv2.imread(image_path)  # Đọc ảnh từ đường dẫn

# Chuyển đổi từ BGR (OpenCV) sang RGB
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV đọc ảnh theo định dạng BGR, chuyển đổi sang RGB

# Hàm tiền xử lý ảnh thành tensor cho YOLO
def preprocess_image(image):
    # Định nghĩa các bước tiền xử lý
    preprocess = transforms.Compose([
        transforms.ToPILImage(),  # Chuyển từ numpy array sang định dạng PIL Image
        transforms.Resize((640, 640)),  # Thay đổi kích thước ảnh về 640x640 pixel
        transforms.ToTensor(),  # Chuyển đổi hình ảnh thành tensor
        # Chuẩn hóa hình ảnh theo chuẩn của ImageNet
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
    ])
    tensor_image = preprocess(image).unsqueeze(0)  # Thêm batch dimension (kích thước 1 cho batch)
    return tensor_image  # Trả về tensor đã được tiền xử lý

In [6]:
# Tải mô hình YOLOv8
model = YOLO('yolov8n.pt')  # Tải mô hình YOLOv8 từ tệp trọng số đã lưu

# Truy cập các lớp backbone
backbone = model.model.model[:10]  # Các lớp từ 0 đến 9 tạo thành backbone của mô hình

# Tạo một mô hình Sequential mới chỉ với các lớp backbone
backbone_model = torch.nn.Sequential(*backbone)  # Kết hợp các lớp backbone thành một mô hình tuần tự

In [7]:
# If you want to use the backbone for feature extraction
def extract_features(image):
    with torch.no_grad():
        features = backbone_model(image)
    return features

In [8]:
# Load data from 220k-GPT4
data = pd.read_parquet("hf://datasets/laion/220k-GPT4Vision-captions-from-LIVIS/lvis_caption_url.parquet")

In [9]:
df = data.head(10000)

In [10]:
# extract features from image
features = {}

for url in tqdm(df['url']):
    response = requests.get(url)
    image_array = np.asarray(bytearray(response.content), dtype=np.uint8)

    # Đọc ảnh bằng OpenCV
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    # Chuyển đổi từ BGR (OpenCV) sang RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Chuyển đổi ảnh thành định dạng tensor phù hợp cho mô hình
    transformed_image = preprocess_image(image_rgb)
    feature_matrix = extract_features(transformed_image)

    # get image ID
    image_id = url

    # store feature
    features[image_id] = feature_matrix


  0%|          | 0/10000 [00:00<?, ?it/s]

In [11]:
# store features in pickle
# pickle.dump(features, open(os.path.join(WORKING_DIR, 'features.pkl'), 'wb'))
file_path = os.path.join(WORKING_DIR, '220k_GPT4_features_10000.pkl')

# Tạo các thư mục nếu chúng chưa tồn tại
os.makedirs(WORKING_DIR, exist_ok=True)

# Lưu dữ liệu vào tệp
with open(file_path, 'wb') as f:
    pickle.dump(features, f)

In [12]:
# # load features from pickle
# with open(os.path.join('/kaggle/input/image-feature-yolov8/220k_GPT4_features_10000.pkl'), 'rb') as f:
#   features = pickle.load(f)

In [13]:
# features['http://images.cocodataset.org/val2017/000000037777.jpg'].size()